In [1]:
import socket
import multiprocessing
import time
from pynq.overlays.base import BaseOverlay
base = BaseOverlay("base.bit")

In [2]:
%%microblaze base.PMODB

#include "gpio.h"
#include "pyprintf.h"

//Function to turn on/off a selected pin of PMODB
unsigned int write_gpio(unsigned int pin, unsigned int val){
    if (val > 1){
        pyprintf("pin value must be 0 or 1");
    }
    else {
        gpio pin_out = gpio_open(pin);
        gpio_set_direction(pin_out, GPIO_OUT);
        gpio_write(pin_out, val);
    }
    return 0;
}

//Function to read the value of a selected pin of PMODB
unsigned int read_gpio(unsigned int pin){
    gpio pin_in = gpio_open(pin);
    gpio_set_direction(pin_in, GPIO_IN);
    return gpio_read(pin_in);
}


# Server code

In [ ]:
SELF_ADDR = '192.168.0.102'
SELF_PORT = 65432
EXT_ADDR  = "192.168.0.100"
EXT_PORT  = 65432
BUF_SIZE = 1024

BUZZ_DURATION = 0.5
BUZZ_PERIOD = 0.01
BUZZ_DUTY_CYCLE = 0.5
BUZZ_DUTY_RATIO = BUZZ_DUTY_CYCLE * BUZZ_PERIOD



def run_server():
    while True:
        sock_l = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock_l.bind((SELF_ADDR, SELF_PORT))
        sock_l.listen()
        print('SERVER: Waiting for connection from client...')
        conn, addr = sock_l.accept()
        print('SERVER: Connected to client.')
        with conn:
            while True:
                data = conn.recv(BUF_SIZE)
                decodeData = data.decode()
                print(f"SERVER: Received messsage {decodeData} from client.")
                if (decodeData == "1"):
                    soundBuzzer()
                elif (decodeData == "2"):
                    break
        print("SERVER: Connection closed.")
        time.sleep(3)

def run_client():
    while True:
        if base.buttons[2].read() != 0:
            print("CLIENT: Attempting to connect to host...")
            with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
                sock.connect((EXT_ADDR, EXT_PORT))
                print("CLIENT: Connected to host.")
                while  True:
                    if base.buttons[0].read() != 0:
                        print("CLIENT: Sending kill to host...")
                        sock.sendall(b"2")
                        time.sleep(0.5)
                        break
                    if base.buttons[1].read() != 0:
                        print("CLIENT: Sending buzz to host...")
                        sock.sendall(b"1")
                        time.sleep(0.5)
            print("Connection closed.")
            time.sleep(4)

def soundBuzzer():
    print("Bzzz!")
    sleepLoops = int(BUZZ_DURATION / BUZZ_PERIOD)
    for i in range(sleepLoops):
        write_gpio(3,1)
        time.sleep(BUZZ_DUTY_RATIO)
        write_gpio(3,0)
        time.sleep(BUZZ_PERIOD-BUZZ_DUTY_RATIO)


p1 = multiprocessing.Process(target=run_server, name="server")
p2 = multiprocessing.Process(target=run_client, name="client")

p1.start()
p2.start()
while(True):
    pass


SERVER: Waiting for connection from client...
CLIENT: Attempting to connect to host...
CLIENT: Connected to host.
CLIENT: Sending buzz to host...
CLIENT: Sending buzz to host...
CLIENT: Sending buzz to host...
SERVER: Connected to client.
SERVER: Received messsage 1 from client.
Bzzz!
SERVER: Received messsage 2 from client.
SERVER: Connection closed.
SERVER: Waiting for connection from client...
SERVER: Connected to client.
SERVER: Received messsage 1 from client.
Bzzz!
SERVER: Received messsage 2 from client.
SERVER: Connection closed.
SERVER: Waiting for connection from client...
SERVER: Connected to client.
SERVER: Received messsage 1 from client.
Bzzz!
SERVER: Received messsage 2 from client.
SERVER: Connection closed.
SERVER: Waiting for connection from client...
CLIENT: Sending kill to host...
Connection closed.


Process server:
Process client:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_1277/446076456.py", line 20, in run_server
    conn, addr = sock_l.accept()
  File "/tmp/ipykernel_1277/446076456.py", line 36, in run_client
    if base.buttons[2].read() != 0:
  File "/usr/lib/python3.10/socket.py", line 293, in accept
    fd, addr = self._accept()
  File "/usr/local/share/pynq-venv/lib/python3.10/site-packages/pynq/lib/axigpio.py", line 49, in read
    return (self._parent.read() >> self._start) & self._mask
KeyboardInt

KeyboardInterrupt: 

# Client code

# Combined multiprocess